In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from haversine import haversine
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import scipy.stats as stats
import folium
import itertools
import statistics
import string

import warnings
warnings.filterwarnings("ignore")

# Naver Map API

In [2]:
# *-- Geocoding 활용 코드 --*
import json
import urllib
from urllib.request import Request, urlopen

In [ ]:
# 주소에 geocoding 적용하는 함수를 작성.
def get_location(loc) :
    # 내 고유값임 (공개안되도록)
    client_id = 'wi1vd6qb31'
    client_secret = 'NWOOthvHMkC11OEqF93vhqXPzsTellMMEq08gtf4'

    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" \
    			+ urllib.parse.quote(loc)
    
    # 주소 변환
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urlopen(request)
    res = response.getcode()
    
    if (res == 200) : # 응답이 정상적으로 완료되면 200을 return
        response_body = response.read().decode('utf-8')
        response_body = json.loads(response_body)
        # print(response_body)
        
        # 주소가 존재할 경우 total count == 1이 반환됨.
        if response_body['meta']['totalCount'] == 1 : 
        	# 위도, 경도 좌표를 받아와서 return해 줌.
            lat = response_body['addresses'][0]['y']
            lon = response_body['addresses'][0]['x']
            return (lon, lat)
        else :
            print('location not exist')
        
    else :
        print('ERROR')

In [ ]:
# *-- Directions 5 활용 코드 --*

# option : 탐색옵션 [최대 3개, traoptimal(기본 옵션)]
# trafast 실시간 빠른길
# tracomfort 실시간 편한길
# traoptimal 실시간 최적
# traavoidtoll 무료 우선
# traavoidcaronly 자동차 전용도로 회피 우선
option = 'traoptimal'

def get_optimal_route(start, goal, option=option ) :

    # 내 고유값임 (공개안되도록)
    client_id = 'wi1vd6qb31'
    client_secret = 'NWOOthvHMkC11OEqF93vhqXPzsTellMMEq08gtf4'

    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    res = response.getcode()
    
    if (res == 200) :
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
            
    else :
        print('ERROR')

In [ ]:
# 주소에서 위도, 경도 뽑고 df 만드는 함수
def geo_df_coding(addresses):
    geo_list = []
    for address in addresses:
        geo_list.append(get_location(address))
        
    geo_df = pd.DataFrame(geo_list)
    geo_df.columns = ['longitude', 'latitude']
    geo_df = geo_df[['latitude', 'longitude']]

    return geo_df, geo_list

In [ ]:
# 도로상 거리, 시간 계산해주는 함수
def dist_time_mat(geo_df, geo_list):
    dist_mat = np.zeros((geo_df.shape[0], geo_df.shape[0]))
    time_mat = np.zeros((geo_df.shape[0], geo_df.shape[0]))

    option = 'traoptimal'
    for i in range(geo_df.shape[0]):
        for j in range(geo_df.shape[0]):
            if (i==j):
                dist_mat[i][j] = 0
                time_mat[i][j] = 0
            else:
                results = get_optimal_route(start = geo_list[i], goal = geo_list[j], option=option)

                dist_mat[i][j] = results['route']['traoptimal'][0]['summary']["distance"]
                time_mat[i][j] = results['route']['traoptimal'][0]['summary']["duration"]
                
    return dist_mat, time_mat

In [3]:
addresses = ['서울시 도봉구 방학로6길 25', 
           '서울시 도봉구 덕릉로63길 19',
           '서울시 도봉구 해등로 133',
           '서울시 도봉구 방학로3길 16',
           '서울시 도봉구 도봉로133길 42',
           '서울시 도봉구 덕릉로59나길 20',
           '서울시 도봉구 해등로16가길 32',
           '서울시 도봉구 우이천로34길 38',
           '서울시 도봉구 노해로 41길 9',
           '서울시 도봉구 도봉로 969',
           '서울특별시 도봉구 마들로 656']

geo_df, geo_list = geo_df_coding(addresses)

dist_mat, time_mat = dist_time_mat(geo_df, geo_list)

# Algorithm